In [1]:
using BenchmarkTools

In [128]:
module sp

import Base: *, +, ^, promote_rule, convert, show

immutable Variable{Name}
end

show(io::IO, v::Variable{Name}) where Name = print(io, Name)

immutable Monomial{Powers}
end

Monomial(v::Variable{Name}) where {Name} = Monomial{((v, 1),)}()

function show(io::IO, m::Monomial{P}) where {P}
    for power in P
        print(io, power[1], "^", power[2])
    end
end

immutable Term{Mono, T}
    coefficient::T
    Term{Mono}(c::T) where{Mono, T} = new{Mono, T}(c)
    Term{Mono, T}(c::T) where{Mono, T} = new{Mono, T}(c)
end

Term(mono::Monomial{Powers}) where Powers = Term{mono}(1)

function show(io::IO, t::Term{Mono, T}) where {Mono, T}
    print(io, t.coefficient, Mono)
end


immutable Polynomial{T <: Tuple}
    terms::T
end

(^)(v::Variable, ::Type{Val{x}}) where x = Monomial{((v, x),)}()
(*)(v1::Variable{Name}, v2::Variable{Name}) where Name = Monomial{((v1, 2),)}()
@generated function (*)(v1::Variable{N1}, v2::Variable{N2}) where {N1, N2}
    @assert N1 != N2
    if N1 < N2
        quote
            Monomial{((v1, 1), (v2, 1))}()
        end
    else
        quote
            Monomial{((v2, 1), (v1, 1))}()
        end
    end
end

(+)(t1::Term{M, T}, t2::Term{M, T}) where {M, T} = Term{M}(t1.coefficient + t2.coefficient)
(+)(m1::Monomial, m2::Monomial) = Term(m1) + Term(m2)
(+)(v1::Variable, v2::Variable) = Term(Monomial(v1)) + Term(Monomial(v2))
(*)(x::Number, t::T) where T <: Term = T(x * t.coefficient)
(*)(x::Number, m::Monomial) = x * Term(m)


end

IJulia.sp

In [129]:
x = sp.Variable{:x}()
y = sp.Variable{:y}()

y

In [130]:
x * x

x^2

In [131]:
x + x

2x^1

In [132]:
x * y

x^1y^1

In [135]:
1 * (x * y) + 3 * (x * y)

4x^1y^1

In [85]:
x^1 + x^1

2x^1

In [68]:
x * y

x^1y^1

In [ ]:
sp.Monomial{}

In [16]:
Base.promote(sp.Power(x), sp.Power(y))

LoadError: [91mUndefVarError: x not defined[39m

In [37]:
@code_warntype x * x

Variables:
  #self#::Base.#*
  v1::IJulia.sp.Variable{:x}
  v2::IJulia.sp.Variable{:x}

Body:
  begin 
      return $(QuoteNode(x^2))
  end::IJulia.sp.Power{x,2}


In [19]:
p = sp.Power{x, 2}()

x^2